In [1]:
import math
import openai
import os
import sklearn.metrics
import random
from tqdm import tqdm
from langchain.llms import OpenAI

In [2]:
import baseten
baseten.login("yiVOous9.2mjkPJSPdya6FUFbGKmXaLyUF6ZxTQYs")

INFO API key set.
INFO:baseten:API key set.


In [3]:
from task_utils import TASKS, load_data, load_prompt, generate_prompts

In [4]:
from baseten.models import FlanT5

model = FlanT5(model_id="Xq69OGP")

gen_kwargs = {
  "temperature": 0.1,
  "repetition_penalty": 1.2,
}

completion = model(
  "Does the clause grant one party an “enterprise,” “all you can eat” or unlimited usage license? Except as the parties may otherwise agree in writing, Converge, to the extent it has the legal right to do so, hereby grants to Vert an irrevocable, perpetual, world-wide, non-exclusive right and license to use, load, store, transmit, execute, copy, market, distribute, in any medium or distribution technology whatsoever, known or unknown, display, perform and sublicense the Converge-Independent Materials and the Third-Party Materials, in both Source Code and Object Code formats, and to make unlimited instantiations thereof, for any and all purposes.",
  max_new_tokens=512,
  early_stopping=True,
  **gen_kwargs
)

In [5]:
completion

['Yes']

In [6]:
## your LLM stack goes here

def call_llm(prompt):
    output =  model(
        prompt,
        max_new_tokens=512,
        early_stopping=True,
        **gen_kwargs
    )
    return output[0]
    

In [7]:
call_llm("Does the clause grant one party an “enterprise,” “all you can eat” or unlimited usage license? Except as the parties may otherwise agree in writing, Converge, to the extent it has the legal right to do so, hereby grants to Vert an irrevocable, perpetual, world-wide, non-exclusive right and license to use, load, store, transmit, execute, copy, market, distribute, in any medium or distribution technology whatsoever, known or unknown, display, perform and sublicense the Converge-Independent Materials and the Third-Party Materials, in both Source Code and Object Code formats, and to make unlimited instantiations thereof, for any and all purposes.")

'Yes'

In [14]:
import json

with open("task_summary_description.json", "r") as f:
    task_summary_description = json.loads(f.read())

In [20]:
def generate_new_prompts(task, data_df):
    summary = task_summary_description[task]["summary"]
    texts = list(data_df["text"])
    prompts = [f'{summary} {text}' for text in texts]
    return prompts

In [21]:
def evaluate(tasks: list, tasks_dir: str):
    report = dict()
    for task in tqdm(tasks):
        train_df, test_df = load_data(task=task, tasks_dir=tasks_dir)
        prompt_template = load_prompt(prompt_name="base_prompt.txt", task=task, tasks_dir=tasks_dir)
        prompts = generate_new_prompts(task=task, data_df=train_df)
        report[task] = dict()
        targets = list()
        outputs = list()
        for prompt, data in zip(prompts, train_df.iterrows()):
            datapoint_id, data = data
            output = call_llm(prompt)
            output = output.strip()
            targets.append(data['answer'])
            outputs.append(output)
            success = output == data['answer']
            report[task][datapoint_id] = {
                'prompt': prompt,
                'generated_output': output,
                'correct_output': data['answer'],
                'success': output == data['answer']
            }
        report[task]['balanced_accuracy'] = sklearn.metrics.balanced_accuracy_score(targets, outputs)
    
    print('Balanced Accuracy:', sum([report[task]['balanced_accuracy'] if not math.isnan(report[task]['balanced_accuracy']) else 0 for task in tasks])/len(tasks))
    
    return report


In [22]:
len(TASKS)

162

In [23]:
tasks_dir = '../legalbench'

# tasks=random.sample(TASKS, 10)

tasks= [task for task in TASKS if task.startswith("cuad")]

In [24]:
len(tasks)

38

In [25]:
report = evaluate(tasks=tasks, tasks_dir=tasks_dir)

100%|██████████| 38/38 [01:28<00:00,  2.33s/it]

Balanced Accuracy: 0.5877192982456142


In [26]:
print('Balanced Accuracy:', sum([report[task]['balanced_accuracy'] if not math.isnan(report[task]['balanced_accuracy']) else 0 for task in tasks])/len(tasks))

Balanced Accuracy: 0.5877192982456142


In [13]:
print('Balanced Accuracy:', sum([report[task]['balanced_accuracy'] if not math.isnan(report[task]['balanced_accuracy']) else 0 for task in tasks])/len(tasks))

Balanced Accuracy: 0.5087719298245613
